# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


![https://s3-us-west-2.amazonaws.com/secure.notion-static.com/069818d1-0e5c-4d87-a461-0de584ab9c33/Untitled_(33).png](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/069818d1-0e5c-4d87-a461-0de584ab9c33/Untitled_(33).png)


### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 
!pip install psycopg2

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
books = pd.read_sql_table('books', engine)
authors = pd.read_sql_table('authors', engine)
publishers = pd.read_sql_table('publishers', engine)
ratings = pd.read_sql_table('ratings', engine)
reviews = pd.read_sql_table('reviews', engine)


Напишем функцию для проверки данных

In [4]:
def data_verification(df):
    print('\nПосмотрим первые 5 строк таблицы')
    display(df.head())
    print(f'\nИмеем таблицу данных размерностью {df.shape}\n')
    print('\nОбщая информация о таблице\n')
    df.info()
    print('\nПроверим наличие дубликатов\n')
    df.duplicated().sum()
    print('\nПроверим наличие пропусков\n')
    df.isna().sum()

***Посмотрим датафреймы из базы данных с которыми мы будем работать***

***books***

In [5]:
data_verification(books)


Посмотрим первые 5 строк таблицы


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Имеем таблицу данных размерностью (1000, 6)


Общая информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   book_id           1000 non-null   int64         
 1   author_id         1000 non-null   int64         
 2   title             1000 non-null   object        
 3   num_pages         1000 non-null   int64         
 4   publication_date  1000 non-null   datetime64[ns]
 5   publisher_id      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 47.0+ KB

Проверим наличие дубликатов


Проверим наличие пропусков



***Пропусков и дубликатов нет***

***authors***

In [6]:
data_verification(authors)


Посмотрим первые 5 строк таблицы


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Имеем таблицу данных размерностью (636, 2)


Общая информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB

Проверим наличие дубликатов


Проверим наличие пропусков



***Пропусков и дубликатов нет.***

***publishers***

In [7]:
data_verification(publishers)


Посмотрим первые 5 строк таблицы


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



Имеем таблицу данных размерностью (340, 2)


Общая информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB

Проверим наличие дубликатов


Проверим наличие пропусков



***Пропусков и дубликатов нет***

***ratings***

In [8]:
data_verification(ratings)


Посмотрим первые 5 строк таблицы


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Имеем таблицу данных размерностью (6456, 4)


Общая информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB

Проверим наличие дубликатов


Проверим наличие пропусков



***Пропусков и дубликатов нет***

***reviews***

In [9]:
data_verification(reviews)


Посмотрим первые 5 строк таблицы


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



Имеем таблицу данных размерностью (2793, 4)


Общая информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB

Проверим наличие дубликатов


Проверим наличие пропусков



***Пропусков и дубликатов нет***

***Вывод***

Имеем пять датафреймов.

Пропусков нет.

Дубликатов нет.

Изменение типа данных не требуется.

Можно приступать к дальнейшему анализу.

# 1. Посчитайте, сколько книг вышло после 1 января 2000 года;

In [10]:
query = "SELECT COUNT(DISTINCT title) FROM books WHERE publication_date>'2000-01-01';"
pd.io.sql.read_sql(query, con = engine) 

,count
0,818


***Вывод***

После `1 января 2000 года` вышло `818 книг`

# 2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [11]:
query= "\
    SELECT b.title,\
    COUNT(DISTINCT rev.review_id) as count_review, \
    AVG(rat.rating) as mean_rating \
    FROM books b \
    LEFT JOIN ratings rat ON b.book_id=rat.book_id \
    LEFT JOIN reviews rev ON b.book_id=rev.book_id \
    GROUP BY b.title \
    ORDER BY count_review DESC;"
pd.io.sql.read_sql(query, con = engine) 

,title,count_review,mean_rating
0,Memoirs of a Geisha,8,4.138462
1,Twilight (Twilight #1),7,3.662500
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Glass Castle,6,4.206897
4,Eat Pray Love,6,3.395833
...,...,...,...
994,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
995,The Natural Way to Draw,0,3.000000
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,Essential Tales and Poems,0,4.000000


***Вывод***

Из сводной таблицы видим, что наличие большого количества отзывов далеко не всегда означает высокие рейтинги.

Есть книги, которые имеют высокий рейтинг, но не имеют ни одного отзыва.

# 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [12]:
query= "\
SELECT \
p.publisher, \
COUNT(b.title) as count_books \
FROM books b \
JOIN publishers p ON b.publisher_id=p.publisher_id \
WHERE num_pages>50 \
GROUP BY p.publisher \
ORDER BY count_books DESC \
LIMIT 1;"
pd.io.sql.read_sql(query, con = engine) 

,publisher,count_books
0,Penguin Books,42


***Вывод***

Издательство `Penguin Books` выпустило больше всех книг, не считая брошюр (толще 50 страниц). 

Всего издательство `Penguin Books` выпустило `42 книги`.

# 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
query= "\
WITH best_author AS \
(SELECT \
r.book_id, \
AVG(r.rating) as mean_rating, \
COUNT(r.rating) as count_rating \
FROM ratings r \
GROUP BY r.book_id \
HAVING COUNT(r.rating) > 49) \
SELECT \
a.author, \
AVG(ba.mean_rating) as mean_rating \
FROM books b \
JOIN authors a ON b.author_id=a.author_id \
JOIN best_author ba ON b.book_id=ba.book_id \
GROUP BY a.author \
ORDER BY AVG(ba.mean_rating) DESC \
LIMIT 1;"
pd.io.sql.read_sql(query, con = engine) 

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.283844


***Вывод***

Самый высокая средняя оценка у `J.K. Rowling/Mary GrandPré`. В среднем ему ставят `4.41`

# 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [14]:
query= "\
WITH \
count_rating AS \
(SELECT \
rat.username, \
COUNT(rat.rating_id) as count_rating \
FROM ratings rat \
GROUP BY rat.username \
HAVING COUNT(rat.rating_id) > 50), \
count_review AS \
(SELECT \
rev.username, \
COUNT(rev.review_id) \
FROM reviews rev \
GROUP BY rev.username) \
SELECT \
ROUND(AVG(crev.count)) as mean_count_rev \
FROM count_rating crat \
JOIN count_review crev ON crat.username=crev.username;"
pd.io.sql.read_sql(query, con = engine) 

,mean_count_rev
0,24.0


***Вывод***

Среднее количество обзоров от пользователей, которые поставили `больше 50 оценок` составило `24.3 обзора`.